In [15]:
# Import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import requests
# Import config file
from config import weatherApiKey
# Import the datetime module from the datetime library.
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
latLngs = zip(lats, lngs)
latLngs

In [3]:
# Covert the latitudes and longitudes from zip object into a list
coordinates = list(latLngs)
coordinates[0:5]

[(18.475477073524218, 139.4434514643786),
 (-41.30877466178399, 114.01105141993935),
 (77.18500195838931, -122.48667905695498),
 (2.55492639999558, -97.00566994763118),
 (-78.48276883610872, -167.664070976894)]

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count
len(cities)

618

In [10]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weatherApiKey

# Create an empty list to hold the weather data.   
cityData = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
recordCount = 1
setCount = 1
# Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposesf
    if (i % 50 == 0 and i >= 50):
        setCount += 1
        recordCount = 1
    # Create endpoint URL with each city
    cityUrl = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {recordCount} of Set {setCount} | {city}")
    # Add 1 to the record count.
    recordCount += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        cityWeather = requests.get(cityUrl).json()
        # Parse out the needed data
        cityLat = cityWeather["coord"]["lat"]
        cityLng = cityWeather["coord"]["lon"]
        cityMaxTemp = cityWeather["main"]["temp_max"]
        cityHumidity = cityWeather["main"]["humidity"]
        cityClouds = cityWeather["clouds"]["all"]
        cityWind = cityWeather["wind"]["speed"]
        cityCountry = cityWeather["sys"]["country"]
        # Convert the date to ISO standard.
        cityDate = datetime.utcfromtimestamp(cityWeather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        cityData.append({
            "City" : city.title(),
            "Lat" : cityLat,
            "Lng" : cityLng,
            "Max Temp" : cityMaxTemp,
            "Humidity" : cityHumidity,
            "Cloudiness" : cityClouds,
            "Wind Speed": cityWind,
            "Country": cityCountry,
            "Date": cityDate})
    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")



of Set 4 | north bend
Processing Record 13 of Set 4 | provideniya
Processing Record 14 of Set 4 | ponta do sol
Processing Record 15 of Set 4 | tavricheskoye
Processing Record 16 of Set 4 | abu jubayhah
City not found. Skipping...
Processing Record 17 of Set 4 | dunedin
Processing Record 18 of Set 4 | hithadhoo
Processing Record 19 of Set 4 | gawler
Processing Record 20 of Set 4 | egvekinot
Processing Record 21 of Set 4 | iqaluit
Processing Record 22 of Set 4 | luderitz
Processing Record 23 of Set 4 | nioro
Processing Record 24 of Set 4 | chengde
Processing Record 25 of Set 4 | baykit
Processing Record 26 of Set 4 | husavik
Processing Record 27 of Set 4 | saint-pierre
Processing Record 28 of Set 4 | san borja
Processing Record 29 of Set 4 | saint anthony
Processing Record 30 of Set 4 | narsaq
Processing Record 31 of Set 4 | cambridge
Processing Record 32 of Set 4 | amapa
Processing Record 33 of Set 4 | sorvag
City not found. Skipping...
Processing Record 34 of Set 4 | grand junction
Pro

In [11]:
# Get the number of the retrieved cities
len(cityData)

571

In [12]:
# Convert the array of dictionaries to a Pandas DataFrame.
cityDataDF = pd.DataFrame(cityData)
cityDataDF.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Airai,-8.93,125.41,70.34,75,59,5.39,TL,2020-07-11 02:45:40
1,Albany,42.60,-73.97,75.99,91,99,6.64,US,2020-07-11 02:43:52
2,Norman Wells,65.28,-126.83,53.60,71,75,13.87,CA,2020-07-11 02:42:06
3,Puerto Ayora,-0.74,-90.35,66.99,95,92,7.38,EC,2020-07-11 02:43:46
4,Vaini,-21.20,-175.20,75.20,78,75,10.29,TO,2020-07-11 02:45:41
5,Bredasdorp,-34.53,20.04,50.00,81,70,16.11,ZA,2020-07-11 02:43:54
6,La Primavera,5.49,-70.41,75.36,90,100,3.47,CO,2020-07-11 02:45:41
7,Mahebourg,-20.41,57.70,68.00,68,40,17.22,MU,2020-07-11 02:45:41
8,Burnie,-41.07,145.92,55.99,69,100,1.01,AU,2020-07-11 02:45:41
9,Tuktoyaktuk,69.45,-133.04,42.80,87,75,12.75,CA,2020-07-11 02:45:41


In [19]:
columnOrder = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
cityDataDF = cityDataDF[columnOrder]
cityDataDF.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Airai,TL,2020-07-11 02:45:40,-8.93,125.41,70.34,75,59,5.39
1,Albany,US,2020-07-11 02:43:52,42.60,-73.97,75.99,91,99,6.64
2,Norman Wells,CA,2020-07-11 02:42:06,65.28,-126.83,53.60,71,75,13.87
3,Puerto Ayora,EC,2020-07-11 02:43:46,-0.74,-90.35,66.99,95,92,7.38
4,Vaini,TO,2020-07-11 02:45:41,-21.20,-175.20,75.20,78,75,10.29
5,Bredasdorp,ZA,2020-07-11 02:43:54,-34.53,20.04,50.00,81,70,16.11
6,La Primavera,CO,2020-07-11 02:45:41,5.49,-70.41,75.36,90,100,3.47
7,Mahebourg,MU,2020-07-11 02:45:41,-20.41,57.70,68.00,68,40,17.22
8,Burnie,AU,2020-07-11 02:45:41,-41.07,145.92,55.99,69,100,1.01
9,Tuktoyaktuk,CA,2020-07-11 02:45:41,69.45,-133.04,42.80,87,75,12.75


In [18]:
# Create the output file (CSV).
outputDataFile = os.path.join("weather_data", "cities.csv")
# Export the City_Data into a CSV.
cityDataDF.to_csv(outputDataFile, index_label="City_ID")
